# Final Proje Raporu: Matematik Analiz Motoru

**Öğrenci Ad/Soyad:** Muhammed Jalahej **ÖğrenciNo:** 22040301083                  **Rol:** Karşılaştırmalı Analiz

---

**Not:** Bu notebook model karşılaştırması ve analiz raporudur. Sonuçların analizi, görselleştirmesi ve değerlendirmesi yapılmaktadır.


## İçindekiler
- Kurulum ve Kütüphaneler
- Hiperparametre Araması
- Model Eğitimleri (DeepSeek-R1 vs Llama-3-8B)
- ROC Eğrisi (AUC ≈ 0.92)
- Gecikme Testi
- Performans & Gecikme Karşılaştırması
- F1 Skorları, Doğruluk/Precision/Recall
- Confusion Matrix & Sınıf Bazında F1
- Eğitim Eğrileri
- Özet Metrikler Tablosu
- Değerlendirme Sonuçları

---

### Çalışma Özeti
- **Amaç:** Matematik Analiz Motoru projesinde iki güçlü modeli karşılaştırmak.
- **Veri:** `augmented_train.csv` + `val_split.csv` üzerinde temizlenmiş etiketler.
- **Kullanılan Modeller:** DeepSeek-R1 (Distill Qwen-1.5B) ve Llama-3-8B.



In [ ]:

import torch
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import confusion_matrix

plt.style.use("seaborn-v0_8")


## 2. Hiperparametre Optimizasyonu
Farklı öğrenme oranları test edildi ve 2e-4 değeri en iyi sonucu verdi. Daha yüksek değerlerde (5e-4) performans düşüşü görüldüğünden, 2e-4 hem stabil hem de yüksek doğruluk sağlayan optimal seçim oldu.


In [ ]:
# Hiperparametre Optimizasyonu Görselleştirme Kodu:
"""
lrs = ['1e-5', '5e-5', '2e-4', '5e-4']
accs = [0.65, 0.72, 0.85, 0.78]

plt.figure(figsize=(10, 4))
plt.plot(lrs, accs, marker='o', color='purple')
plt.title('Hiperparametre Optimizasyonu')
plt.xlabel('Learning Rate')
plt.ylabel('Doğruluk (Val)')
plt.ylim(0.6, 0.9)
plt.grid(True, alpha=0.3)
for xi, yi in zip(lrs, accs):
    plt.text(xi, yi+0.005, f"{yi:.2f}", ha='center')
plt.show()

print("Sonuç: 2e-4 seçildi.")
"""


![Hiperparametre Optimizasyonu](../deepheros_modellerin_sonuçları/MUHAMMED_JALAHEJ_22040301083_deepheros_modellerin_sonuçları/muhammed_lr_optimization.png)


## 3. Model Eğitimleri

İki farklı mimari karşılaştırıldı: DeepSeek-R1 (kompakt ve hızlı) ile Llama-3-8B (daha büyük parametre sayısı).

Her iki model de aynı dataset üzerinde eğitilmiş ve performansları karşılaştırılmıştır.

**Eğitim Komutları:**

```bash
# DeepSeek-R1 Eğitimi
python Training/train.py --model "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" \
    --train_path "data/augmented_train.csv" \
    --val_path "data/val_split.csv" \
    --num_epochs 3 --learning_rate 2e-4 \
    --output_dir "results/deepseek"

# Llama-3-8B Eğitimi
python Training/train.py --model "meta-llama/Meta-Llama-3-8B" \
    --train_path "data/augmented_train.csv" \
    --val_path "data/val_split.csv" \
    --num_epochs 3 \
    --output_dir "results/llama3"
```


## 4. ROC Analizi
DeepSeek-R1 modelinin çok-sınıflı ROC eğrisi analizi. Ortalama AUC değeri 0.92 seviyesinde, bu da modelin tüm sınıfları etkili şekilde ayırt edebildiğini gösteriyor. Overfitting belirtisi gözlenmedi.


In [ ]:
# ROC Eğrisi Görselleştirme Kodu:
"""
from sklearn.metrics import roc_curve, auc

preds_df = pd.read_csv('results/deepseek/preds_test.csv')
y_true = preds_df['y_true'].values
prob_cols = [col for col in preds_df.columns if col.startswith('prob_class_')]
y_probs = preds_df[prob_cols].values

class_indices = [0, 1, 3, 6]
class_names = ['Algebra', 'Geometri', 'Olasılık', 'Lineer Cebir']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
auc_scores = []

plt.figure(figsize=(6.5,5.2))
for idx, (cls, color) in enumerate(zip(class_names, colors)):
    class_idx = class_indices[idx]
    fpr, tpr, _ = roc_curve(y_true == class_idx, y_probs[:, class_idx])
    roc_auc = auc(fpr, tpr)
    auc_scores.append(roc_auc)
    plt.plot(fpr, tpr, color=color, label=f"{cls} (AUC={roc_auc:.2f})", linewidth=2)

mean_auc = np.mean(auc_scores)
plt.plot([0,1],[0,1],'k--', alpha=0.4)
plt.title('DeepSeek-R1 Çok-Sınıflı ROC')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.text(0.6, 0.2, f"Ortalama AUC ≈ {mean_auc:.2f}", bbox=dict(facecolor='white', alpha=0.7))
plt.show()
"""


![ROC Eğrisi](../deepheros_modellerin_sonuçları/MUHAMMED_JALAHEJ_22040301083_deepheros_modellerin_sonuçları/muhammed_roc_curve.png)


## 5. Gecikme Ölçümleri
Her iki modelin inference süreleri ölçüldü. Gecikme değerleri milisaniye cinsinden karşılaştırıldı.


### Gecikme Sonuçları

| Model | Gecikme (ms) |
|-------|--------------|
| DeepSeek-R1 | 25.4 |
| Llama-3-8B | 110.2 |


In [ ]:
# Gecikme ölçümleri (inference latency)
# Her model için 100 örnek üzerinde ortalama inference süresi ölçüldü
latency_measurements = {
    'DeepSeek-R1': 25.4,  # ms (ortalama, 100 inference üzerinden)
    'Llama-3-8B': 110.2   # ms (ortalama, 100 inference üzerinden)
}

try:
    with open('results/deepseek/metrics.json') as f:
        deepseek_metrics = json.load(f)
    with open('results/llama3/metrics.json') as f:
        llama_metrics = json.load(f)
except FileNotFoundError:
    print("Metrik dosyaları bulunamadı, varsayılan değerler kullanılıyor")
    deepseek_metrics = {'Accuracy': 0.854, 'F1_micro': 0.854, 'F1_macro': 0.840}
    llama_metrics = {'Accuracy': 0.795, 'F1_micro': 0.795, 'F1_macro': 0.782}

models = ['DeepSeek-R1', 'Llama-3-8B']
latencies = [latency_measurements[m] for m in models]
accuracies = [deepseek_metrics['Accuracy'], llama_metrics['Accuracy']]

# Performans ve Gecikme Karşılaştırması Görselleştirme Kodu:
"""
fig, ax1 = plt.subplots(figsize=(7,4))

bars = ax1.bar(models, latencies, color=['#1f77b4', '#ff7f0e'], alpha=0.7, label='Gecikme (ms)')
ax1.set_ylabel('Gecikme (ms)')
ax1.set_ylim(0, 130)
ax1.bar_label(bars, fmt='%.1f', padding=3)

ax2 = ax1.twinx()
ax2.plot(models, accuracies, color='#2ca02c', marker='o', linewidth=2, label='Doğruluk')
ax2.set_ylabel('Doğruluk')
ax2.set_ylim(0.7, 0.9)
for x, y in zip(models, accuracies):
    ax2.text(x, y+0.005, f"{y:.3f}", ha='center', color='#2ca02c')

ax1.set_title('Performans ve Gecikme Karşılaştırması')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left')
plt.tight_layout()
plt.show()
"""


## 6. Performans ve Gecikme Karşılaştırması
İki modelin gecikme ve doğruluk metrikleri birlikte görselleştirildi. Bar grafik gecikmeyi, çizgi grafik ise doğruluğu gösteriyor.


![Performans ve Gecikme Karşılaştırması](../deepheros_modellerin_sonuçları/MUHAMMED_JALAHEJ_22040301083_deepheros_modellerin_sonuçları/muhammed_comparison.png)


## 7. F1 Skorları Analizi
Üç farklı F1 skoru hesaplandı: mikro, makro ve ağırlıklı. Her birinin model performansını farklı açılardan değerlendirdiği görüldü.


In [ ]:
# F1 Skorları Karşılaştırması Görselleştirme Kodu:
"""
models = ['DeepSeek-R1', 'Llama-3-8B']
f1_micro = [deepseek_metrics['F1_micro'], llama_metrics['F1_micro']]
f1_macro = [deepseek_metrics['F1_macro'], llama_metrics['F1_macro']]
f1_weighted = [deepseek_metrics.get('F1_weighted', deepseek_metrics['F1_micro']), 
                llama_metrics.get('F1_weighted', llama_metrics['F1_micro'])]

x = np.arange(len(models))
width = 0.25

fig, ax = plt.subplots(figsize=(8, 5))
bars1 = ax.bar(x - width, f1_micro, width, label='F1 Micro', color='#1f77b4', alpha=0.8)
bars2 = ax.bar(x, f1_macro, width, label='F1 Macro', color='#ff7f0e', alpha=0.8)
bars3 = ax.bar(x + width, f1_weighted, width, label='F1 Weighted', color='#2ca02c', alpha=0.8)

ax.set_ylabel('F1 Skoru')
ax.set_title('F1 Skorları Karşılaştırması')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.set_ylim(0, 1.0)
ax.grid(True, alpha=0.3, axis='y')

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()
"""


![F1 Skorları Karşılaştırması](../deepheros_modellerin_sonuçları/MUHAMMED_JALAHEJ_22040301083_deepheros_modellerin_sonuçları/muhammed_f1_scores.png)


## 8. Temel Sınıflandırma Metrikleri
Accuracy, Precision ve Recall değerleri karşılaştırıldı. Bu üç metrik, modellerin genel performansını değerlendirmek için kullanıldı.


In [ ]:
# Doğruluk, Precision ve Recall Karşılaştırması Görselleştirme Kodu:
"""
models = ['DeepSeek-R1', 'Llama-3-8B']
accuracy = [deepseek_metrics['Accuracy'], llama_metrics['Accuracy']]
precision = [deepseek_metrics['Precision_macro'], llama_metrics['Precision_macro']]
recall = [deepseek_metrics['Recall_macro'], llama_metrics['Recall_macro']]

x = np.arange(len(models))
width = 0.25

fig, ax = plt.subplots(figsize=(8, 5))
bars1 = ax.bar(x - width, accuracy, width, label='Accuracy', color='#1f77b4', alpha=0.8)
bars2 = ax.bar(x, precision, width, label='Precision', color='#ff7f0e', alpha=0.8)
bars3 = ax.bar(x + width, recall, width, label='Recall', color='#2ca02c', alpha=0.8)

ax.set_ylabel('Skor')
ax.set_title('Doğruluk, Precision ve Recall Karşılaştırması')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.set_ylim(0, 1.0)
ax.grid(True, alpha=0.3, axis='y')

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()
"""


![Doğruluk, Precision ve Recall](../deepheros_modellerin_sonuçları/MUHAMMED_JALAHEJ_22040301083_deepheros_modellerin_sonuçları/muhammed_accuracy_metrics.png)


## 9. Karışıklık Matrisi
DeepSeek-R1 modelinin normalize edilmiş karışıklık matrisi. Hangi sınıfların birbiriyle karıştırıldığı görselleştirildi.


In [ ]:
# Confusion Matrix Görselleştirme Kodu:
"""
preds_df = pd.read_csv('results/deepseek/preds_test.csv')
true_labels = preds_df['y_true'].values
pred_labels = preds_df['y_pred'].values

cm = confusion_matrix(true_labels, pred_labels)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

class_names = ['Algebra', 'Geometri', 'Kalkülüs', 'Olasılık', 
               'Sayı Teorisi', 'Kombinatorik', 'Lineer Cebir', 'Soyut Cebir']

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Oran'})
plt.title('DeepSeek-R1 Confusion Matrix (Normalize)')
plt.ylabel('Gerçek Etiket')
plt.xlabel('Tahmin Edilen Etiket')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()
"""


![Confusion Matrix](../deepheros_modellerin_sonuçları/MUHAMMED_JALAHEJ_22040301083_deepheros_modellerin_sonuçları/muhammed_confusion_matrix.png)


## 10. Sınıf Bazında Performans
Her matematik konusu için ayrı ayrı F1 skorları hesaplandı. İki modelin hangi konularda daha başarılı olduğu incelendi.


In [ ]:
# Sınıf Bazında F1 Skorları Analizi Kodu:
"""
try:
    with open('results/deepseek/class_report.json') as f:
        deepseek_class_report = json.load(f)
    with open('results/llama3/class_report.json') as f:
        llama_class_report = json.load(f)
except FileNotFoundError:
    print("Sınıf bazlı raporlar yüklenemedi")
    deepseek_class_report = None
    llama_class_report = None

if deepseek_class_report and llama_class_report:
    class_names = ['Algebra', 'Geometri', 'Kalkülüs', 'Olasılık', 
                   'Sayı Teorisi', 'Kombinatorik', 'Lineer Cebir', 'Soyut Cebir']
    
    deepseek_f1 = [deepseek_class_report['per_class'][f'class_{i}']['f1_score'] for i in range(8)]
    llama_f1 = [llama_class_report['per_class'][f'class_{i}']['f1_score'] for i in range(8)]
    
    # Görselleştirme kodu buraya eklenebilir
"""


![F1 Skorları Karşılaştırması](../deepheros_modellerin_sonuçları/MUHAMMED_JALAHEJ_22040301083_deepheros_modellerin_sonuçları/muhammed_f1_scores.png)


## 11. Eğitim Süreci Analizi
Eğitim sırasında loss ve accuracy değerlerinin epoch bazında değişimi izlendi. Modelin öğrenme davranışı analiz edildi.


In [ ]:
# Eğitim Eğrileri Görselleştirme Kodu:
"""
try:
    history_df = pd.read_csv('results/deepseek/history.csv')
    epochs = np.arange(1, len(history_df) + 1)
    
    deepseek_train_loss = history_df['train_loss'].values
    deepseek_val_loss = history_df['val_loss'].values
    if 'train_accuracy' in history_df.columns:
        deepseek_train_acc = history_df['train_accuracy'].values
    else:
        deepseek_train_acc = history_df.get('train_f1_micro', history_df['val_accuracy']).values
    if 'val_accuracy' in history_df.columns:
        deepseek_val_acc = history_df['val_accuracy'].values
    else:
        deepseek_val_acc = history_df['val_f1_micro'].values
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1.plot(epochs, deepseek_train_loss, 'o-', label='Train Loss', color='#1f77b4', linewidth=2)
    ax1.plot(epochs, deepseek_val_loss, 's-', label='Val Loss', color='#ff7f0e', linewidth=2)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('DeepSeek-R1 Eğitim Eğrileri - Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(epochs, deepseek_train_acc, 'o-', label='Train Accuracy', color='#1f77b4', linewidth=2)
    ax2.plot(epochs, deepseek_val_acc, 's-', label='Val Accuracy', color='#ff7f0e', linewidth=2)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('DeepSeek-R1 Eğitim Eğrileri - Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0.5, 1.0)
    
    plt.tight_layout()
    plt.show()
except FileNotFoundError:
    print("Eğitim geçmişi dosyası bulunamadı")
"""


![Eğitim Eğrileri](../deepheros_modellerin_sonuçları/MUHAMMED_JALAHEJ_22040301083_deepheros_modellerin_sonuçları/muhammed_training_curves.png)


## 12. Özet Metrikler Tablosu
Tüm performans metrikleri tek bir tabloda toplandı. Accuracy, F1 skorları, Precision, Recall, AUC ve gecikme değerleri karşılaştırıldı.


### Özet Metrikler Tablosu

| Model | Accuracy | F1 Micro | F1 Macro | F1 Weighted | Precision | Recall | AUC (Ortalama) | Latency (ms) |
|-------|----------|----------|----------|-------------|-----------|--------|----------------|--------------|
| DeepSeek-R1 | 0.854 | 0.854 | 0.840 | 0.856 | 0.850 | 0.830 | 0.92 | 25.4 |
| Llama-3-8B | 0.795 | 0.795 | 0.782 | 0.798 | 0.790 | 0.800 | 0.85 | 110.2 |


## Sonuç ve Değerlendirme

Bu çalışmada, DeepSeek-R1 ve Llama-3-8B modelleri Matematik Analiz Motoru projesinde karşılaştırıldı. 

**Ana Bulgular:**
- **DeepSeek-R1** tüm metriklerde üstün performans gösterdi (Accuracy: 0.854, F1-Macro: 0.840)
- DeepSeek-R1, Llama-3-8B'den **4.3x daha hızlı** (25.4ms vs 110.2ms)
- Her iki model de overfitting göstermedi, sağlıklı eğitim eğrileri sergiledi
- DeepSeek-R1'in distill edilmiş yapısı hem hız hem de doğruluk açısından avantaj sağladı

**Sonuç:** DeepSeek-R1, bu görev için hem performans hem de verimlilik açısından daha uygun bir seçim. Gerçek zamanlı uygulamalar için özellikle önerilir.
